# Function Calling with Gemini 

**Learning Objectives**

1. Learn about function calling and relevant use cases
1. Learn how to implement function calling with Gemini Pro
1. Learn patterns for handling function calls in a chat session
1. Learn how function calling can be used in different situations and use cases 

Function calling allows developers to define custom functions and provide these functions to Gemini. While processing a query, Gemini can choose to delegate certain data processing tasks to these functions. Gemini does not call these functions, rather it provides structured data output that includes the name of a selected function and the arguments the function should be called with. You can use this output to perform tasks like invoking external APIs, performing mathematical computations, extracting structured data, and more. You can then provide the function response back to the model, allowing it to complete its answer to the query.

<img src="https://cloud.google.com/static/vertex-ai/generative-ai/docs/multimodal/images/function-calling.png" alt="Function Calling" class="center">

In [ ]:
from typing import Any, Callable, Optional, Tuple, Union

from google import genai
from google.cloud import bigquery
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GenerateContentResponse,
    Part,
    Schema,
    Tool,
)

In [ ]:
REGION = "us-central1"
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]

MODEL = "gemini-2.0-flash-001"

client = genai.Client(vertexai=True, location="us-central1")

## Chat Session with Function Calling
First, lets think about how function calling can be implemented within a chat session. Essentially, when the model returns a function call, instead of returning to the user, we need to invoke a Python function that executes the specified function with the provided arguments, then feeds the result back into the model. This may happen multiple times (e.g. model returns function call -> function call response fed back into model -> model returns another function call -> ... ). 

Our goal is to create a simple class for example chat sessions, that implements a reasoning loop in its `send_message` method. The class should be instantiated with:
1) `tools`: A list of `Tool`s
2) `tool_handler_fn`. A Python callable that accepts the function call name (str) and the function call arguments (dict) when invoked. This function should implement the logic of the function call itself and return the result.

For example, if we had a tool with function calls for reading from and writing to a database, then we may have a `tool_handler_fn` that looks like:

```python
def tool_handler_fn(fn_name, fn_args):
    """This assumes function call read_row has parameter row_id, and function call write_row has parameter row"""
    if fn_name == "read_row":
        result = db.read_row(fn_args["row_id"])
    elif fn_name == "write_row":
        result = db.write_row(fn_args["row"])
    return result 
```

In [ ]:
class ChatAgent:
    def __init__(
        self,
        tools: list[Tool],
        tool_handler_fn: Callable[[str, dict], Any],
        max_iterative_calls: int = 5,
    ):
        self.tools = tools
        self.tool_handler_fn = tool_handler_fn
        self.chat_session = client.chats.create(
            model=MODEL,
            config=GenerateContentConfig(tools=tools),
        )
        self.max_iterative_calls = 5

    def send_message(self, message: str) -> GenerateContentResponse:
        response = self.chat_session.send_message(message)
        # This is None if a function call was not triggered
        fn_calls = response.function_calls

        num_calls = 0
        # Reasoning loop. If fn_calls is empty then we never enter this
        # and simply return the response
        while fn_calls:
            if num_calls > self.max_iterative_calls:
                break

            # Handle the function calls
            fn_call_responses = []
            for fn_call in fn_calls:
                response = self.tool_handler_fn(
                    fn_call.name, dict(fn_call.args)
                )
                fn_call_responses.append(
                    Part.from_function_response(
                        name=fn_call.name,
                        response={
                            "content": response,
                        },
                    ),
                )
                num_calls += 1

            # Send the function call result back to the model
            response = self.chat_session.send_message(fn_call_responses)

            # If the response is another function call then we want to
            # stay in the reasoning loop and keep calling functions.
            fn_calls = response.function_calls

        return response

## Simple API Example
Now that we have a way to use function calling in a chat session, let's implement some common use cases. Imagine you want to call an API to get the current weather for a specific location, when a user asks for it. This requires a mechanism to identify that the current weather is being asked for, and also to extract the location required in the API request. 

With function calling, this is fairly straightforward. Simply define a function declaration with the intent and required parameters.

In [ ]:
current_weather_func = FunctionDeclaration(
    name="current_weather",
    description="Get the current weather at a specified location",
    parameters=Schema(
        type="OBJECT",
        properties={
            "location": Schema(
                type="STRING",
                description="Location",
            ),
        },
        required=["location"],
    ),
)

# Simulate a function that calls a weather API


def current_weather(location: str) -> dict:
    print("Executing current_weather function...")
    api_response = {
        "location": "New York City",
        "temperature": "55 degrees (F)",
        "wind": "8 mph",
        "wind_direction": "West",
        "skies": "clear/sunny",
        "chance_of_rain": "0%",
    }
    return api_response

Instantiate a `Tool` with the single function declaration, and then write a tool handler function to invoke when the model returns a function call. Then instantiate the model with the `Tool`.

In [ ]:
# Tools can wrap around one or multiple functions
weather_tool = Tool(
    function_declarations=[current_weather_func],
)

# Instantiate chat with weather tool
chat = client.chats.create(
    model=MODEL, config=GenerateContentConfig(tools=[weather_tool])
)

Send a chat through the model without using `ChatAgent` to see what the response of a function call looks like.

In [ ]:
response = chat.send_message("What is the weather like in New York City?")
response

In [ ]:
response.function_calls

Notice how instead of returning a text response, Gemini returned the function name to call and arguments to call it with. Now implement a function that we can instantiate `ChatAgent` with, that we will pass the function name and arguments to any time Gemini returns a function call.

In [ ]:
def weather_tool_handler_fn(fn_name: str, fn_args: dict) -> dict:
    if fn_name == "current_weather":
        return current_weather(fn_args["location"])
    else:
        raise ValueError(f"Unknown function call: {fn_name}")


chat = ChatAgent(tools=[weather_tool], tool_handler_fn=weather_tool_handler_fn)
response = chat.send_message("What is the weather like in New York City?")
response.text

If we take a look at the chat history, we can see that a function call was returned, our handler function was invoked, which then invoked the Python function simulating an API. The response from that was then sent back into the model and incorporated in its response about the weather!

In [ ]:
chat.chat_session.get_history()

## Function calling to perform mathematical operations
Function calling can also help in an area that LLMs have long struggled - mathematics. Language models build up deep and insightful representations of natural language, but often lack the ability to (correctly and consistently) perform mathematical operations. We can provide a degree of consistency and accuracy by creating a tool that identifies when a mathematical operation is needed, and calls a function to actually perform that operation. 

Create function declarations for simple mathematical operations (addition, subtraction, multiplication, division).

#### Exercise
Implement the four mathematical functions to invoke when a function call is triggered. You need to write functions to add, subtract, multiply and divide numbers.

In [ ]:
parameters = Schema(
    type="OBJECT",
    properties={
        "first_number": Schema(
            type="NUMBER",
            description="First number",
        ),
        "second_number": Schema(
            type="NUMBER",
            description="Second_number number",
        ),
    },
    required=["first_number", "second_number"],
)


# TODO: Create function declarations for core math operations
add_two_numbers_func = None
subtract_two_numbers_func = None
multiply_two_numbers_func = None
divide_two_numbers_func = None

math_tool = Tool(
    function_declarations=[
        add_two_numbers_func,
        subtract_two_numbers_func,
        multiply_two_numbers_func,
        divide_two_numbers_func,
    ],
)

Instead of simulating the response from functions, lets actually write the Python functions that we will call with arguments provided when Gemini responds with a function call.

In [ ]:
# Define functions for each function declaration used in the math tool
add_two_numbers = lambda a, b: a + b
subtract_two_numbers = lambda a, b: a - b
multiply_two_numbers = lambda a, b: a * b
divide_two_numbers = lambda a, b: a / b

#### Exercise 

Implement a handler using if else statemnets to route function calls

In [ ]:
def handle_math_fn_call(fn_name: str, fn_args: dict) -> Union[int, float]:
    """Handles math tool function calls."""

    print(f"Function calling: {fn_name} with args: {fn_args}")
    a = fn_args["first_number"]
    b = fn_args["second_number"]

    # TODO: Complete this function to handle different function calls

**Exercise:** Instantiate a model, chat agent, and test out some queries!

In [ ]:
chat = ChatAgent(
    tools=[None], tool_handler_fn=handle_math_fn_call  # TODO: add the tools
)

In [ ]:
response = chat.send_message("What is one plus one?")
response.text

In [ ]:
response = chat.send_message("Thanks! What is (5 * 5) / (4 + 1)?")
response.text

Notice how Gemini called more than one function, sequentially and logically, in order to answer the question. Very cool! 

## Natural Language to SQL with Database Execution 
Function calling can be helpful with systems that require SQL generation and execution, and using the response to answer a query. Start by creating a dataset in BigQuery and cloning some public tables into it.

In [ ]:
# Create the dataset
!bq mk --location="US" iowa_liquor_sales

Create the table by querying the public data

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE iowa_liquor_sales.sales AS 
SELECT * FROM `bigquery-public-data.iowa_liquor_sales.sales`

Update the schema of your table to include column definitions

In [ ]:
SCHEMA_FILE = "liquor_sales_schema.json"
!bq show --schema --format=prettyjson bigquery-public-data:iowa_liquor_sales.sales > {SCHEMA_FILE}
!bq update {PROJECT}:iowa_liquor_sales.sales {SCHEMA_FILE}

Create function declarations and instantiate a new `Tool`. The function declarations should be:
1) Listing available tables `list_available_tables`
2) Retrieving information and schema about a specific table `get_table_info`
3) Retrieves information from BigQuery to answer a users question `sql_query`

#### Exercise 
1. Implement the `sql_query_func` function declaration with one parameter "query". Provide a description for this query so that it only executes the query if the table is in the project.
2. Implement a tool that gathers all the function declarations 

In [ ]:
# Since we only have one table we will just hardcode the response from this function if triggered
list_available_tables_func = FunctionDeclaration(
    name="list_available_tables",
    description="Get and list all available BigQuery tables with fully qualified IDs.",
    parameters=Schema(type="OBJECT", properties={}),
)

get_table_info_func = FunctionDeclaration(
    name="get_table_info",
    description="Get information about a BigQuery table and it's schema so you can better answer user questions.",
    parameters=Schema(
        type="OBJECT",
        properties={
            "table_id": Schema(
                type="STRING",
                description="Fully qualified ID of BigQuery table",
            )
        },
    ),
)

sql_query_func = FunctionDeclaration(
    name="sql_query",
    description="Get information from data in BigQuery using SQL queries",
    parameters=None,  # TODO: Define one parameter "query". Provide a description for this query so that it only executes the query if the table is in the project.
)

# TODO: Instantiate the query tool using the above function declarations
query_tool = None

Now we need to create Python functions that will be executed when the model returns any of these function calls. The functions should be implement as such:
* `list_available_tables` should accept no parameters and simply return the name of the BigQuery table created above: `f"{PROJECT}/iowa_liquor_sales.sales"`
* `get_table_info` should accept a table_id parameter and use the BigQuery client library to retrieve table information and schema 
* `sql_query` should accept a query_sting parameter and use the BigQuery client library to execute a sql query and return the results

In [ ]:
def list_available_tables():
    return [f"{PROJECT}.iowa_liquor_sales.sales"]


def get_table_info(table_id: str) -> dict:
    """Returns dict from BigQuery API with table information"""
    bq_client = bigquery.Client()
    return bq_client.get_table(table_id).to_api_repr()


def sql_query(query_str: str):
    bq_client = bigquery.Client()
    try:
        # clean up query string a bit
        query_str = (
            query_str.replace("\\n", "").replace("\n", "").replace("\\", "")
        )
        # print(query_str)
        query_job = bq_client.query(query_str)
        result = query_job.result()
        result = str([dict(x) for x in result])
        return result
    except Exception as e:
        return f"Error from BigQuery Query API: {str(e)}"

**Exercise:** Create a Python function to handle function calls returned by the model and invoke the needed logic. Use if else statements to map the functions.

In [ ]:
def handle_query_fn_call(fn_name: str, fn_args: dict):
    """Handles query tool function calls."""

    print(f"Function calling: {fn_name} with args: {str(fn_args)}\n")
    # TODO: If-else statements to map the fn_name to the right function call

    return result

Instantiate a model, chat agent, and test out some queries!

In [ ]:
chat = ChatAgent(tools=[query_tool], tool_handler_fn=handle_query_fn_call)

In [ ]:
# Insert an initialization prompt before the first chat to help guide model behavior and output style/format

init_prompt = """
    Please give a concise and easy to understand answer to any questions. 
    Only use information that you learn by querying the BigQuery table. 
    Do not make up information. Be sure to look at which tables are available 
    and get the info of any relevant tables before trying to write a query. 
    
    Question:
"""

prompt = "Which store has sold the most bottles of all time?"
response = chat.send_message(init_prompt + prompt)
print(response.text)

In [ ]:
response = chat.send_message(
    "Interesting! What is the most popular bottle of all time?"
)
print(response.text)

In [ ]:
response = chat.send_message(
    "What are the five most popular bottles in polk county?"
)
print(response.text)

In [ ]:
response = chat.send_message(
    "What vendors have made the most revenue selling liquor?"
)
print(response.text)

Feel free to execute the generated SQL to verify/validate the responses! An easy way to do this is in a code cell with `%%bigquery` at the top. For example:

```
%%bigquery
SELECT ... 
FROM ... 
```

## Function calling for entity extraction 
In the previous examples we used entity extraction to pass parameters along to another function, API, or client library. However, you might want to only perform the entity extraction step, and stop there without actually invoking anything else. You can think of this functionality as a convenient way to transform unstructured text data into structured fields.

For example, we can easily build a log extractor that transforms raw logs into structured data with details about error messages.

Start by specifying the function declaration.

In [ ]:
extract_log_data_func = FunctionDeclaration(
    name="extract_log_data",
    description="Extracts specific details from errors in log data",
    parameters=Schema(
        type="OBJECT",
        properties={
            "errors": Schema(
                type="ARRAY",
                description="Errors",
                items=Schema(
                    type="OBJECT",
                    description="Details of the error",
                    properties={
                        "error_message": Schema(
                            type="STRING",
                            description="Full error message",
                        ),
                        "error_code": Schema(
                            type="STRING",
                            description="Error code",
                        ),
                        "error_type": Schema(
                            type="STRING",
                            description="Error type",
                        ),
                    },
                ),
            ),
        },
    ),
)

error_extraction_tool = Tool(
    function_declarations=[extract_log_data_func],
)

In [ ]:
prompt = """
[15:43:28] ERROR: Could not process image upload: Unsupported file format. (Error Code: 308)
[15:44:10] INFO: Search index updated successfully. 
[15:45:02] ERROR: Service dependency unavailable (payment gateway). Retrying... (Error Code: 5522) 
[15:45:33] ERROR: Application crashed due to out-of-memory exception. (Error Code: 9001) 
"""

response = client.models.generate_content(
    model=MODEL,
    config=GenerateContentConfig(tools=[error_extraction_tool]),
    contents=prompt,
)
function_calls = response.function_calls

In [ ]:
for func_call in function_calls:
    for err in dict(func_call.args).get("errors"):
        print(dict(err))

Function calling is an incredibly versatile tool! 

## (Optional) Function Calling with LangChain
Let's see how we can implement function calling using LangChain.

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_google_vertexai import ChatVertexAI

chat = ChatVertexAI(model="gemini-2.0-flash")

To implement function calling in LangChain, you utilize LangChain `tools`. These tools are defined by schemas, which can be created from various Python constructs: regular functions (including type hints and docstrings), [Pydantic models](https://python.langchain.com/docs/how_to/tool_calling/#pydantic-class), [TypedDict classes](https://python.langchain.com/docs/how_to/tool_calling/#typeddict-class), or dedicated [LangChain Tool objects](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.base.BaseTool.html#basetool) instances. The following example illustrates how to define a tool using a Python function, each function is decorated with the `@tool` decorator."

In [ ]:
@tool
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


@tool
def subtract(a: int, b: int) -> int:
    """Subtract second_number from first_number.

    Args:
        a: First integer
        b: Second integer
    """
    return a - b


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b


@tool
def divide(a: int, b: int) -> int:
    """Divide first_number by second_number.

    Args:
        a: First integer
        b: Second integer
    """
    return a / b

In Python function, you can further customize it by adding `@tool` decorator. Please check [the document](https://python.langchain.com/docs/how_to/custom_tools/#tool-decorator) for more details.

Now let's bind the tools with `.bind_tools()` method.

In [ ]:
tools = [add, subtract, multiply, divide]

chat_with_tools = chat.bind_tools(tools)

Now you can invoke the model and see the response from the model.<br>
The response contains the `.tool_calls` when function calling is requested by the model.

Here we manage the chat history in `message` list, but please consider using LangGraph when building a complex agent system. 

In [ ]:
query = "What is (5 * 5) / (4 + 1)?"
messages = [HumanMessage(query)]

response = chat_with_tools.invoke(messages)
messages.append(response)
response.tool_calls

Let's call each function and appned to the `message`. Please note the response from tools are wrapped in a [`ToolMessage`](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.tool.ToolMessage.html) object.

In [ ]:
for tool_call in response.tool_calls:
    selected_tool = {
        "add": add,
        "subtract": subtract,
        "multiply": multiply,
        "divide": divide,
    }[tool_call["name"]]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

Now we provide the responses from functions to the LLM model and call it again.

In [ ]:
response = chat_with_tools.invoke(messages)
messages.append(response)
response.tool_calls

The model responded with another function call. Let's repeat the same process again!

In [ ]:
for tool_call in response.tool_calls:
    selected_tool = {
        "add": add,
        "subtract": subtract,
        "multiply": multiply,
        "divide": divide,
    }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

chat_with_tools.invoke(messages)

Now we successfully got the final response!

Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.